# Tuning and Optimizing Neural Networks - Lab

## Introduction

Now that we've discussed some regularization, initialization and optimization techniques, its time to synthesize those concepts into a cohesive modelling pipeline.  

With this pipeline, yoiu will not only fit an initial model but will also attempt to set various hyperparameters for regularization techniques. Your final model selection will pertain to the test metrics across these models. This will more naturally simulate a problem you might be faced with in practice, and the various modelling decisions you are apt to encounter along the way.  

Recall that our end objective is to achieve a balance between overfitting and underfitting. We've discussed the bias variance tradeoff, and the role of regularization in order to reduce overfitting on training data and improving generalization to new cases. Common frameworks for such a procedure include train/validate/test methodology when data is plentiful, and K-folds cross-validation for smaller, more limited datasets. In this lab, you'll perform the latter, as the dataset in question is fairly limited. 

## Objectives

You will be able to:

* Implement a K-folds cross validation modelling pipeline
* Apply normalization as a preprocessing technique
* Apply regularization techniques to improve your model's generalization
* Choose an appropriate optimization strategy 

## Loading the Data

In [1]:
#Your code here; load and preview the dataset
!ls

CONTRIBUTING.md  index.ipynb  Loan_clean.csv  README.md
index_files	 LICENSE.md   loan_final.csv


In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('Loan_clean.csv', nrows=10)
df_ = pd.read_csv('loan_final.csv', nrows=10)
df.columns

Index(['loan_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment',
       'grade', 'emp_length', 'home_ownership', 'annual_inc',
       'verification_status',
       ...
       'Unnamed: 95', 'Unnamed: 96', 'Unnamed: 97', 'Unnamed: 98',
       'Unnamed: 99', 'Unnamed: 100', 'Unnamed: 101', 'Unnamed: 102',
       'Unnamed: 103', 'Unnamed: 104'],
      dtype='object', length=105)

In [3]:
df_.columns

Index(['loan_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment',
       'grade', 'emp_length', 'home_ownership', 'annual_inc',
       'verification_status', 'loan_status', 'purpose', 'addr_state',
       'total_acc', 'total_pymnt', 'application_type'],
      dtype='object')

In [4]:
df = pd.read_csv('loan_final.csv')
print(df.shape)
df.head()

(42538, 16)


,loan_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,addr_state,total_acc,total_pymnt,application_type
0,5000.0,4975.0,36 months,10.65%,162.87,B,10+ years,RENT,24000.0,Verified,Fully Paid,credit_card,AZ,9.0,5863.155187,Individual
1,2500.0,2500.0,60 months,15.27%,59.83,C,< 1 year,RENT,30000.0,Source Verified,Charged Off,car,GA,4.0,1014.530000,Individual
2,2400.0,2400.0,36 months,15.96%,84.33,C,10+ years,RENT,12252.0,Not Verified,Fully Paid,small_business,IL,10.0,3005.666844,Individual
3,10000.0,10000.0,36 months,13.49%,339.31,C,10+ years,RENT,49200.0,Source Verified,Fully Paid,other,CA,37.0,12231.890000,Individual
4,3000.0,3000.0,60 months,12.69%,67.79,B,1 year,RENT,80000.0,Source Verified,Fully Paid,other,OR,38.0,4066.908161,Individual


## Defining the Problem

Set up the problem by defining X and Y. 

For this problem use the following variables for X:
* loan_amnt
* home_ownership
* funded_amnt_inv
* verification_status
* emp_length
* installment
* annual_inc

Be sure to use dummy variables for categorical variables and to normalize numerical quanitities. Be sure to also remove any rows with null data.  

For Y, we are looking to build a model to predict the total payment received for a loan.

In [5]:
# Your code here; appropriately define X and Y using dummy variables and normalization for preprocessing.
raw = df[['loan_amnt', 'home_ownership', 'funded_amnt_inv', 'verification_status',
            'emp_length', 'installment', 'annual_inc', 'total_pymnt']]

In [6]:
raw.head()

,loan_amnt,home_ownership,funded_amnt_inv,verification_status,emp_length,installment,annual_inc,total_pymnt
0,5000.0,RENT,4975.0,Verified,10+ years,162.87,24000.0,5863.155187
1,2500.0,RENT,2500.0,Source Verified,< 1 year,59.83,30000.0,1014.530000
2,2400.0,RENT,2400.0,Not Verified,10+ years,84.33,12252.0,3005.666844
3,10000.0,RENT,10000.0,Source Verified,10+ years,339.31,49200.0,12231.890000
4,3000.0,RENT,3000.0,Source Verified,1 year,67.79,80000.0,4066.908161


In [7]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42538 entries, 0 to 42537
Data columns (total 8 columns):
loan_amnt              42535 non-null float64
home_ownership         42535 non-null object
funded_amnt_inv        42535 non-null float64
verification_status    42535 non-null object
emp_length             41423 non-null object
installment            42535 non-null float64
annual_inc             42531 non-null float64
total_pymnt            42535 non-null float64
dtypes: float64(5), object(3)
memory usage: 2.6+ MB


In [8]:
raw.isna().sum()

loan_amnt                 3
home_ownership            3
funded_amnt_inv           3
verification_status       3
emp_length             1115
installment               3
annual_inc                7
total_pymnt               3
dtype: int64

In [9]:
raw[raw.loan_amnt.isna()]

,loan_amnt,home_ownership,funded_amnt_inv,verification_status,emp_length,installment,annual_inc,total_pymnt
39786,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39787,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39788,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# DROP IT LIKE A BAD HABIT
print(raw.shape)
raw = raw.drop(df[df.loan_amnt.isna()].index)
raw.shape

(42538, 8)


(42535, 8)

In [11]:
raw.isna().sum()

loan_amnt                 0
home_ownership            0
funded_amnt_inv           0
verification_status       0
emp_length             1112
installment               0
annual_inc                4
total_pymnt               0
dtype: int64

In [12]:
display(raw[raw.annual_inc.isna()])
print(f'Shape: {raw.shape}')

,loan_amnt,home_ownership,funded_amnt_inv,verification_status,emp_length,installment,annual_inc,total_pymnt
42452,5000.0,NONE,3025.0,Not Verified,< 1 year,155.38,NaN,5593.631614
42453,7000.0,NONE,3450.0,Not Verified,< 1 year,218.55,NaN,7867.655865
42483,6700.0,NONE,6700.0,Not Verified,< 1 year,209.18,NaN,7530.481427
42536,6500.0,NONE,0.0,Not Verified,< 1 year,204.84,NaN,7373.904962


Shape: (42535, 8)


In [13]:
# Just 4 out of 40k - - Drip drop
print(raw.shape)
raw = raw.drop(raw[raw.annual_inc.isna()].index)
print(raw.shape)

(42535, 8)
(42531, 8)


In [14]:
raw.isna().sum()

loan_amnt                 0
home_ownership            0
funded_amnt_inv           0
verification_status       0
emp_length             1112
installment               0
annual_inc                0
total_pymnt               0
dtype: int64

In [15]:
display(raw.emp_length.value_counts())
print(f'- - About as many nans as nine-year exps. Lets just bin the nans - -')

10+ years    9369
< 1 year     5058
2 years      4743
3 years      4364
4 years      3649
1 year       3595
5 years      3458
6 years      2375
7 years      1875
8 years      1592
9 years      1341
Name: emp_length, dtype: int64

- - About as many nans as nine-year exps. Lets just bin the nans - -


In [16]:
for col in raw:
    if raw[col].dtype == 'object':
        print(f"{col} has values:\n\t{raw[col].unique()}")

home_ownership has values:
	['RENT' 'OWN' 'MORTGAGE' 'OTHER' 'NONE']
verification_status has values:
	['Verified' 'Source Verified' 'Not Verified']
emp_length has values:
	['10+ years' '< 1 year' '1 year' '3 years' '8 years' '9 years' '4 years'
 '5 years' '6 years' '2 years' '7 years' nan]


In [17]:
pd.get_dummies(raw, dummy_na=True).columns

Index(['loan_amnt', 'funded_amnt_inv', 'installment', 'annual_inc',
       'total_pymnt', 'home_ownership_MORTGAGE', 'home_ownership_NONE',
       'home_ownership_OTHER', 'home_ownership_OWN', 'home_ownership_RENT',
       'home_ownership_nan', 'verification_status_Not Verified',
       'verification_status_Source Verified', 'verification_status_Verified',
       'verification_status_nan', 'emp_length_1 year', 'emp_length_10+ years',
       'emp_length_2 years', 'emp_length_3 years', 'emp_length_4 years',
       'emp_length_5 years', 'emp_length_6 years', 'emp_length_7 years',
       'emp_length_8 years', 'emp_length_9 years', 'emp_length_< 1 year',
       'emp_length_nan'],
      dtype='object')

In [18]:
raw = pd.get_dummies(raw, dummy_na=True)

In [19]:
print(raw.shape)
raw.isna().sum()

(42531, 27)


loan_amnt                              0
funded_amnt_inv                        0
installment                            0
annual_inc                             0
total_pymnt                            0
home_ownership_MORTGAGE                0
home_ownership_NONE                    0
home_ownership_OTHER                   0
home_ownership_OWN                     0
home_ownership_RENT                    0
home_ownership_nan                     0
verification_status_Not Verified       0
verification_status_Source Verified    0
verification_status_Verified           0
verification_status_nan                0
emp_length_1 year                      0
emp_length_10+ years                   0
emp_length_2 years                     0
emp_length_3 years                     0
emp_length_4 years                     0
emp_length_5 years                     0
emp_length_6 years                     0
emp_length_7 years                     0
emp_length_8 years                     0
emp_length_9 yea

In [20]:
for col in raw:
    if raw[col].dtype == "float64":
        x = raw[col]
        raw[col] = (x - x.mean()) / x.std()

In [21]:
raw.iloc[:,:6].describe()

,loan_amnt,funded_amnt_inv,installment,annual_inc,total_pymnt,home_ownership_MORTGAGE
count,4.253100e+04,4.253100e+04,4.253100e+04,4.253100e+04,4.253100e+04,42531.000000
mean,5.346069e-18,3.100720e-16,-2.539383e-17,-1.450121e-16,-1.603821e-17,0.445769
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.497056
min,-1.428954e+00,-1.421892e+00,-1.469200e+00,-1.049054e+00,-1.321646e+00,0.000000
25%,-7.947731e-01,-7.278057e-01,-7.519856e-01,-4.545743e-01,-7.209572e-01,0.000000
50%,-1.875789e-01,-2.300265e-01,-2.150201e-01,-1.581456e-01,-2.569342e-01,0.000000
75%,5.275610e-01,5.411807e-01,5.056164e-01,2.084899e-01,4.846287e-01,1.000000
max,3.226202e+00,3.485790e+00,4.702720e+00,9.253044e+01,5.153038e+00,1.000000


In [52]:
X = raw.drop('total_pymnt', axis=1)
y = raw[['total_pymnt']]

## Generating a Hold Out Test Set

While we will be using K-fold cross validation to select an optimal model, we still want a final hold out test set that is completely independent of any modelling decisions. As such, pull out a sample of 10% of the total available data. For consistency of results, use random seed 123. 

In [53]:
# Your code here; generate a hold out test set for final model evaluation. Use random seed 123.
from sklearn.model_selection import train_test_split

X_train, X_holdout, y_train, y_holdout = train_test_split(X,y, test_size = 0.1, random_state=123)

## Defining a K-fold Cross Validation Methodology

Now that your have a complete holdout test set, write a function that takes in the remaining data and performs k-folds cross validation given a model object. Be sure your function returns performance metrics regarding the training and validation sets.

In [60]:
#Your code here; define a function to evaluate a model object using K folds cross validation.

def k_folds(features_train, labels_train, model_obj, k=10, n_epochs=100):
    losses = []
    for i in range(k):
        print(f'{i} started')
        N = len(features_train)
        start_index = round(i * N / k)
        stop_index = round((i + 1) * N / k) + 1
        
        y_val = labels_train.iloc[start_index: stop_index]
        
        y = pd.concat((labels_train.iloc[:start_index], (labels_train.iloc[stop_index:])))
        
        X_val = features_train.iloc[start_index: stop_index]
        
        X = pd.concat((features_train.iloc[:start_index], features_train.iloc[stop_index:]))
        
        print(i)
        print('yval', y_val.shape)
        print('y', y.shape)
        print('xval', X_val.shape)
        print('x', X.shape)
        print('\n')
        
        hist = model_obj.fit(X, y, batch_size=32, epochs=n_epochs,
                     validation_data=(X_val, y_val), verbose=0)
        loss = hist.history['val_loss']
        losses.append(loss)
        print(f'{i} done - - loss: {loss}\n')
    return losses

## Building a Baseline Model

Here, it is also important to define your evaluation metric that you will look to optimize while tuning the model.   

In general, model training to optimize this metric may consist of using a validation and test set if data is plentiful, or k-folds cross-validation if data is limited. We set up a k-folds cross-validation for this task since the dataset is not overly large.  

Build an initial sequential model with 2 hidden relu layers. The first should have 7 hidden units, and the second 10 hidden units. Finally, add a third layer with a linear activation function to output our predictions for the total loan payment. 

In [55]:
#Your code here; define and compile an initial model as described
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(7, input_dim = 26, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='linear'))

In [56]:
model.compile(optimizer='sgd', loss='mse',)

## Evaluating the Baseline Model with K-Folds Cross Validation

Use your k-folds function to evaluate the baseline model.  

Note: This code block is likely to take 10-20 minutes to run depending on the specs on your computer.
Because of time dependencies, it can be interesting to begin timing these operations for future reference.

Here's a simple little recipe to achieve this:
```
import time
import datetime

now = datetime.datetime.now()
later = datetime.datetime.now()
elapsed = later - now
print('Time Elapsed:', elapsed)
```

In [57]:
#Your code here; use your k-folds function to evaluate the baseline model.
import datetime
from datetime import timezone

In [58]:
prev = datetime.datetime.now()
current = datetime.datetime.now()
current - prev

datetime.timedelta(0, 0, 75)

In [46]:
model.fit(X_train.iloc[:-4000],y_train.iloc[:-4000],batch_size=32, epochs=5,
          validation_data=(X_train.iloc[-4000:], y_train[-4000:]))

Train on 34277 samples, validate on 4000 samples
Epoch 1/5
34277/34277 [==============================] - 2s 51us/step - loss: 0.1920 - val_loss: 0.1993
Epoch 2/5
34277/34277 [==============================] - 2s 56us/step - loss: 0.1911 - val_loss: 0.2000
Epoch 3/5
34277/34277 [==============================] - 2s 49us/step - loss: 0.1907 - val_loss: 0.1988
Epoch 4/5
34277/34277 [==============================] - 2s 46us/step - loss: 0.1902 - val_loss: 0.1967
Epoch 5/5
34277/34277 [==============================] - 2s 46us/step - loss: 0.1894 - val_loss: 0.1960


In [61]:
prev = datetime.datetime.now()
losses = k_folds(features_train=X_train, labels_train=y_train, model_obj=model, k=2)
current = datetime.datetime.now()
current - prev

0 started
0
yval (19139, 1)
y (19138, 1)
xval (19139, 26)
x (19138, 26)


0 done - - loss: [0.22009661412946332, 0.20461524385291086, 0.20257015882600166, 0.19761635282285797, 0.20247547748202321, 0.19555709790608727, 0.19717222683984625, 0.21858682051208353, 0.19378041468575877, 0.19384065598807673, 0.19448092023938532, 0.26207057967648884, 0.19364101406868267, 0.19309462031325306, 0.19558744078242307, 0.19227389079279253, 0.19232099204208325, 0.20494865760984973, 0.1921794418893913, 0.19179398198412767, 0.19266641675916746, 0.19162887725376668, 0.1914468863089946, 0.19185367099653872, 0.19176846825003319, 0.19211462328887932, 0.19133658008817825, 0.19088881828100698, 0.1911219942640739, 0.19073246407634356, 0.19059373526278972, 0.19107448997103413, 0.19160418450813682, 0.1912925181689183, 0.19078633004523188, 0.19010419820495345, 0.18967990573172322, 0.19033711535421197, 0.189545959499732, 0.1894975509622035, 0.19179354136144705, 0.18937908768329983, 0.19040519335444453, 0.1891546610

datetime.timedelta(0, 246, 580578)

## Intentionally Overfitting a Model

Now that you've developed a baseline model, its time to intentionally overfit a model. To overfit a model, you can:
* Add layers
* Make the layers bigger
* Increase the number of training epochs

Again, be careful here. Think about the limitations of your resources, both in terms of your computers specs and how much time and patience you have to let the process run. Also keep in mind that you will then be regularizing these overfit models, meaning another round of experiments and more time and resources.  

For example, here are some timing notes on potential experiments run on a Macbook Pro 3.1 GHz Intel Core i5 with 16gb of RAM:

* Using our 10 fold cross validation methodology, a 5-layer neural network with 10 units per hidden layer and 100 epochs took approximately 15 minutes to train and validate  

* Using our 10 fold cross validation methodology, a 5-layer neural network with 25 units per hidden layer and 100 epochs took approximately 25 minutes to train and validate  

* Using our 10 fold cross validation methodology, a 5-layer neural network with 10 units per hidden layer and 250 epochs took approximately 45 minutes to train and validate


In [ ]:
#Your code here; try some methods to overfit your network

In [ ]:
#Your code here; try some methods to overfit your network

In [ ]:
#Your code here; try some methods to overfit your network

## Regularizing the Model to Achieve Balance  

Now that you have a powerful model (albeit an overfit one), we can now increase the generalization of the model by using some of the regularization techniques we discussed. Some options you have to try include:  
* Adding dropout
* Adding L1/L2 regularization
* Altering the layer architecture (add or remove layers similar to above)  

This process will be constrained by time and resources. Be sure to test at least 2 different methodologies, such as dropout and L2 regularization. If you have the time, feel free to continue experimenting.

Notes: 

In [ ]:
#Your code here; try some regularization or other methods to tune your network

In [ ]:
#Your code here; try some regularization or other methods to tune your network

In [ ]:
#Your code here; try some regularization or other methods to tune your network

In [ ]:
#Your code here; try some regularization or other methods to tune your network

## Final Evaluation

Now that you have selected a network architecture, tested various regularization procedures and tuned hyperparameters via a validation methodology, it is time to evaluate your finalized model once and for all. Fit the model using all of the training and validation data using the architecture and hyperparameters that were most effective in your expirements above. Afterwards, measure the overall performance on the hold-out test data which has been left untouched (and hasn't leaked any data into the modelling process)!

In [ ]:
#Your code here; final model training on entire training set followed by evaluation on hold-out data

## Additional Resources

https://machinelearningmastery.com/dropout-regularization-deep-learning-models-keras/

https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/

https://machinelearningmastery.com/regression-tutorial-keras-deep-learning-library-python/

https://stackoverflow.com/questions/37232782/nan-loss-when-training-regression-network
https://www.springboard.com/blog/free-public-data-sets-data-science-project/

## Summary

In this lab, we investigated some data from *The Lending Club* in a complete data science pipeline regarding neural networks. We began with reserving a hold-out set for testing which never was touched during the modeling phase. From there, we implemented a k-fold cross validation methodology in order to assess an initial baseline model and various regularization methods. From here, we'll begin to investigate other neural network architectures such as CNNs.